### Google Maps API Script 2 - Obtaining the cuisines, ratings, and the price level.

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import json
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import scipy
import sys
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import emoji
import requests
import googlemaps
import warnings

In [3]:
# reading in api key and loading this in
f = open("api.txt", "r")

gmaps = googlemaps.Client(key=f.readlines()[1])
f.close()

In [4]:
restaurant_IDs = pd.read_csv('Restaurant_Location_Details.csv')

In [ ]:
cocotte_test = gmaps.place(place_id = 'ChIJJeZ74wEQdkgRRaSxXA9ggsM',
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')

In [16]:
brewery_test = gmaps.place(place_id = 'ChIJqdYI34sPdkgRvMVH8LGc4Mw',
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')

In [18]:
# print(cocotte_test)
print(brewery_test)

{'html_attributions': [], 'result': {}, 'status': 'OK'}


In [13]:
dict_list = [cocotte_test['result'], brewery_test['result']]

df = pd.DataFrame()
for i in range(2):
    dict_selected = dict_list[i]
    df = df.append(dict_selected, ignore_index=True)
print(df)



                                   editorial_summary  price_level  rating  \
0  {'language': 'en', 'overview': 'Welcoming, rus...            2     4.4   
1  {'language': 'en', 'overview': 'Gastropub serv...            2     4.2   

   user_ratings_total  
0                 740  
1                1619  


C:\Users\ndsna\AppData\Local\Temp\ipykernel_37436\2145559040.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)
C:\Users\ndsna\AppData\Local\Temp\ipykernel_37436\2145559040.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dict_selected, ignore_index=True)


In [5]:
def extract_values(dictionary):
    overview_lang = dictionary['language']
    overview = dictionary['overview']
    return overview_lang, overview
df[['overview_lang', 'overview']] = df['editorial_summary'].apply(lambda x: pd.Series(extract_values(x)))

NameError: name 'df' is not defined

In [6]:
restaurant_list = restaurant_IDs['place_id']


In [20]:
def extract_values(dictionary):
    if dictionary == dictionary:
        overview_lang = dictionary['language']
        overview = dictionary['overview']
        return overview_lang, overview

missing_restaurants = []

with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    df = pd.DataFrame()
    for i in restaurant_list:
        restaurant_details = gmaps.place(place_id = i,
                                fields = ['price_level','rating','user_ratings_total','editorial_summary'], #,'primaryTypeName','primaryTypeDisplayName'
                                language = 'en',
                                reviews_no_translations = False, # translating reviews into English here for now
                                reviews_sort = 'newest')
        # moving past any case where it cannot be found
        if restaurant_details['result']:
            dict_details = restaurant_details['result']
            dict_details["place_id"] = i
            df = df.append(dict_details, ignore_index=True)

        # else:
        #     missing_restaurants = missing_restaurants.append(i)

    df[['overview_lang', 'overview']] = df['editorial_summary'].apply(lambda x: pd.Series(extract_values(x)))
    print(df)




                                      editorial_summary  price_level  rating  \
0     {'language': 'en', 'overview': 'Welcoming, rus...          2.0     4.4   
1     {'language': 'en', 'overview': 'A carefully-cu...          NaN     4.7   
2     {'language': 'en', 'overview': 'Charming Greek...          2.0     4.6   
3                                                   NaN          NaN     4.8   
4     {'language': 'en', 'overview': 'Inmates prep a...          2.0     4.8   
...                                                 ...          ...     ...   
1780  {'language': 'en', 'overview': 'A marble featu...          3.0     4.2   
1781  {'language': 'en', 'overview': 'Modern Italian...          2.0     3.8   
1782  {'language': 'en', 'overview': 'Hotel's upscal...          NaN     4.0   
1783  {'language': 'en', 'overview': 'Upbeat, party-...          2.0     4.2   
1784  {'language': 'en', 'overview': 'Gastropub serv...          2.0     4.2   

      user_ratings_total               

In [14]:
print(dict_details)

{'editorial_summary': {'language': 'en', 'overview': 'Sophisticated decor with tiles and exposed bricks in a daytime weekend cafe and nighttime bar.'}, 'price_level': 2, 'rating': 4.8, 'user_ratings_total': 826}


In [21]:
df.to_csv('Restaurant_Location_Additional_Details.csv', index = False)